In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np
import os
import sys

In [3]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [4]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data\\notebooks'

In [5]:
os.chdir("..")

In [6]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

In [7]:
os.listdir()

['.virtual_documents',
 'Community',
 'data_processed',
 'data_raw',
 'desktop.ini',
 'notebooks',
 'site']

In [8]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

In [9]:
# -----------------------------
# 2. Sample Data Preparation
# -----------------------------
os.chdir("C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data\\data_processed\\Pickel files")
df_2024 = pd.read_pickle("df_2024.pkl")

In [10]:
os.chdir("C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data")

In [11]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

#### Data import

In [12]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime

In [13]:
df_2024.shape

(44303209, 13)

In [14]:
df_2024.shape

(44303209, 13)

In [15]:
# df_2024 = df_2024[:10000]

In [16]:
df_2024.shape

(44303209, 13)

#### Main routes - Option1 - color intensity

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import requests
import time

from dotenv import load_dotenv
import os

load_dotenv()
ors_api_key = os.getenv('ORS_API_KEY')
mapbox_api_key = os.getenv('MAPBOX_API_KEY')




# Your OpenRouteService API key
ORS_API_KEY = ors_api_key  # Replace with your API key

def get_route_coordinates(start_lng, start_lat, end_lng, end_lat):
    """Get cycling route from OpenRouteService API"""
    headers = {
        'Authorization': ORS_API_KEY,
        'Content-Type': 'application/json; charset=utf-8'
    }
    
    body = {"coordinates": [[start_lng, start_lat], [end_lng, end_lat]],
            "profile": "cycling-regular",
            "format": "geojson"}
    
    try:
        response = requests.post(
            'https://api.openrouteservice.org/v2/directions/cycling-regular/geojson',
            json=body,
            headers=headers
        )
        
        if response.status_code == 200:
            route = response.json()
            return route['features'][0]['geometry']['coordinates']
        else:
            print(f"Error getting route: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

def get_top_routes(df_2024, top_n=300):
    routes = df_2024.groupby(
        ['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
    ).size().reset_index(name='rides')
    
    routes = routes[routes['start_station_name'] != routes['end_station_name']]
    return routes.nlargest(top_n, 'rides')

# Get top 50 routes
top_routes = get_top_routes(df_2024, top_n=300)

# Normalize rides for color scaling
top_routes['normalized_rides'] = (top_routes['rides'] - top_routes['rides'].min()) / \
                                (top_routes['rides'].max() - top_routes['rides'].min())

# Create figure
fig = go.Figure()

# Add routes with actual paths
for idx, route in top_routes.iterrows():
    coords = get_route_coordinates(
        route['start_lng'], 
        route['start_lat'], 
        route['end_lng'], 
        route['end_lat']
    )
    
    if coords:
        lngs, lats = zip(*coords)
        
        # Calculate color based on number of rides
        color_intensity = 0.3 + (route['normalized_rides'] * 2.0)  # ranges from 0.3 to 0.9
        
        fig.add_trace(go.Scattermapbox(
            mode='lines',
            lon=list(lngs),
            lat=list(lats),
            line=dict(
                width=3,
                color=f'rgba(99, 171, 255, {color_intensity})'  # Uber-style blue
            ),
            hovertemplate=(
                f"<b>Route {idx+1} of 50</b><br>" +
                f"From: {route['start_station_name']}<br>" +
                f"To: {route['end_station_name']}<br>" +
                f"Number of rides: {route['rides']:,}<br>" +
                "<extra></extra>"
            ),
            showlegend=False
        ))
    
    # Add a small delay to avoid hitting API rate limits
    time.sleep(1)

# Add station points
stations = pd.concat([
    df_2024[['start_station_name', 'start_lat', 'start_lng']].rename(
        columns={'start_station_name': 'station_name', 'start_lat': 'lat', 'start_lng': 'lng'}
    ),
    df_2024[['end_station_name', 'end_lat', 'end_lng']].rename(
        columns={'end_station_name': 'station_name', 'end_lat': 'lat', 'end_lng': 'lng'}
    )
]).drop_duplicates()

fig.add_trace(go.Scattermapbox(
    lat=stations['lat'],
    lon=stations['lng'],
    mode='markers',
    marker=dict(
        size=4,
        color='white',
        opacity=0.8
    ),
    showlegend=False,
    hoverinfo='none'
))

# Update layout
fig.update_layout(
    mapbox_style="carto-darkmatter",
    mapbox=dict(
        center=dict(lat=40.7505, lon=-73.9934),
        zoom=11.5
    ),
    margin={"r":0,"t":50,"l":0,"b":0},
    title={
        'text': "Most Popular Citibike Routes (2024)<br>" +
               "<sup>Color intensity indicates number of rides</sup>",
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(color='blue', size=16)
    },
    paper_bgcolor='#1a1f35',
    plot_bgcolor='#1a1f35'
)

# Save the map
fig.write_html("site/charts/top_routes_map.html")

# Print some statistics
print(f"Most popular route: {top_routes.iloc[0]['start_station_name']} → {top_routes.iloc[0]['end_station_name']}")
print(f"Number of rides on most popular route: {top_routes.iloc[0]['rides']:,}")
print(f"Minimum rides shown: {top_routes.iloc[-1]['rides']:,}")


Most popular route: River Ter & Warren St → Vesey Pl & River Terrace
Number of rides on most popular route: 7,367
Minimum rides shown: 3,687


### Main routes - Option2 - color intensity

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import requests
import time

# Your OpenRouteService API key
ORS_API_KEY = ors_api_key  # Replace with your API key

def get_route_coordinates(start_lng, start_lat, end_lng, end_lat):
    """Get cycling route from OpenRouteService API"""
    headers = {
        'Authorization': ORS_API_KEY,
        'Content-Type': 'application/json; charset=utf-8'
    }
    
    body = {"coordinates": [[start_lng, start_lat], [end_lng, end_lat]],
            "profile": "cycling-regular",
            "format": "geojson"}
    
    try:
        response = requests.post(
            'https://api.openrouteservice.org/v2/directions/cycling-regular/geojson',
            json=body,
            headers=headers
        )
        
        if response.status_code == 200:
            route = response.json()
            return route['features'][0]['geometry']['coordinates']
        else:
            print(f"Error getting route: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

def get_top_routes(df_2024, top_n=300):
    routes = df_2024.groupby(
        ['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
    ).size().reset_index(name='rides')
    
    routes = routes[routes['start_station_name'] != routes['end_station_name']]
    return routes.nlargest(top_n, 'rides')

# Get top 50 routes
top_routes = get_top_routes(df_2024, top_n=300)

# Normalize rides for color scaling
top_routes['normalized_rides'] = (top_routes['rides'] - top_routes['rides'].min()) / \
                                (top_routes['rides'].max() - top_routes['rides'].min())

# Create figure
fig = go.Figure()

# Add routes with actual paths
for idx, route in top_routes.iterrows():
    coords = get_route_coordinates(
        route['start_lng'], 
        route['start_lat'], 
        route['end_lng'], 
        route['end_lat']
    )
    
    if coords:
        lngs, lats = zip(*coords)
        
        # Calculate color based on number of rides
        color_intensity = 0.3 + (route['normalized_rides'] * 2.0)  # ranges from 0.3 to 0.9
        
        fig.add_trace(go.Scattermapbox(
            mode='lines',
            lon=list(lngs),
            lat=list(lats),
            line=dict(
                width=3,
                color=f'rgba(99, 171, 255, {color_intensity})'  # Uber-style blue
            ),
            hovertemplate=(
                f"<b>Route {idx+1} of 50</b><br>" +
                f"From: {route['start_station_name']}<br>" +
                f"To: {route['end_station_name']}<br>" +
                f"Number of rides: {route['rides']:,}<br>" +
                "<extra></extra>"
            ),
            showlegend=False
        ))
    
    # Add a small delay to avoid hitting API rate limits
    time.sleep(1)

# Add station points
stations = pd.concat([
    df_2024[['start_station_name', 'start_lat', 'start_lng']].rename(
        columns={'start_station_name': 'station_name', 'start_lat': 'lat', 'start_lng': 'lng'}
    ),
    df_2024[['end_station_name', 'end_lat', 'end_lng']].rename(
        columns={'end_station_name': 'station_name', 'end_lat': 'lat', 'end_lng': 'lng'}
    )
]).drop_duplicates()

fig.add_trace(go.Scattermapbox(
    lat=stations['lat'],
    lon=stations['lng'],
    mode='markers',
    marker=dict(
        size=4,
        color='white',
        opacity=0.8
    ),
    showlegend=False,
    hoverinfo='none'
))

# Previous code remains the same until the layout section

# Update layout with modified title and added colorbar
fig.update_layout(
    mapbox_style="carto-darkmatter",
    mapbox=dict(
        center=dict(lat=40.7505, lon=-73.9934),
        zoom=11.5
    ),
    margin={"r":0,"t":50,"l":0,"b":0},
    title={
        'text': "Most Popular Citibike Routes (2024)",
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(color='#333333', size=16)  # Dark colored title
    },
    paper_bgcolor='#1a1f35',
    plot_bgcolor='#1a1f35'
)

# Add a separate trace for the colorbar
fig.add_trace(go.Scattermapbox(
    lat=[None],
    lon=[None],
    mode='markers',
    marker=dict(
        size=0,
        colorscale=[
            [0, 'rgba(99, 171, 255, 0.3)'],
            [1, 'rgba(99, 171, 255, 0.9)']
        ],
        colorbar=dict(
            title='Number of Rides',
            titleside='right',
            x=1.02,
            xanchor='left',
            titlefont=dict(color='white'),
            tickfont=dict(color='white'),
            thickness=15
        ),
        cmin=top_routes['rides'].min(),
        cmax=top_routes['rides'].max(),
        showscale=True
    ),
    showlegend=False,
    hoverinfo='none'
))

# Rest of the code remains the same


# Save the map
fig.write_html("site/charts/top_routes_map.html")

# Print some statistics
print(f"Most popular route: {top_routes.iloc[0]['start_station_name']} → {top_routes.iloc[0]['end_station_name']}")
print(f"Number of rides on most popular route: {top_routes.iloc[0]['rides']:,}")
print(f"Minimum rides shown: {top_routes.iloc[-1]['rides']:,}")


Most popular route: River Ter & Warren St → Vesey Pl & River Terrace
Number of rides on most popular route: 7,367
Minimum rides shown: 2,082


### Main routes - Option3 - color intensity

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import requests
import time

# Your OpenRouteService API key
ORS_API_KEY = ors_api_key  # Replace with your API key

def get_route_coordinates(start_lng, start_lat, end_lng, end_lat):
    """Get cycling route from OpenRouteService API"""
    headers = {
        'Authorization': ORS_API_KEY,
        'Content-Type': 'application/json; charset=utf-8'
    }
    
    body = {"coordinates": [[start_lng, start_lat], [end_lng, end_lat]],
            "profile": "cycling-regular",
            "format": "geojson"}
    
    try:
        response = requests.post(
            'https://api.openrouteservice.org/v2/directions/cycling-regular/geojson',
            json=body,
            headers=headers
        )
        
        if response.status_code == 200:
            route = response.json()
            return route['features'][0]['geometry']['coordinates']
        else:
            print(f"Error getting route: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

def get_top_routes(df_2024, top_n=400):
    routes = df_2024.groupby(
        ['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
    ).size().reset_index(name='rides')
    
    routes = routes[routes['start_station_name'] != routes['end_station_name']]
    return routes.nlargest(top_n, 'rides')

# Get top 50 routes
top_routes = get_top_routes(df_2024, top_n=400)

# Normalize rides for color scaling
top_routes['normalized_rides'] = (top_routes['rides'] - top_routes['rides'].min()) / \
                                (top_routes['rides'].max() - top_routes['rides'].min())

# Create figure
fig = go.Figure()

# Add routes with actual paths
for idx, route in top_routes.iterrows():
    coords = get_route_coordinates(
        route['start_lng'], 
        route['start_lat'], 
        route['end_lng'], 
        route['end_lat']
    )
    
    if coords:
        lngs, lats = zip(*coords)
        
        # Calculate color based on number of rides
        color_intensity = 0.3 + (route['normalized_rides'] * 2.0)  # ranges from 0.3 to 0.9
        
        fig.add_trace(go.Scattermapbox(
            mode='lines',
            lon=list(lngs),
            lat=list(lats),
            line=dict(
                width=3,
                color=f'rgba(99, 171, 255, {color_intensity})'  # Uber-style blue
            ),
            hovertemplate=(
                f"<b>Route {idx+1} of 50</b><br>" +
                f"From: {route['start_station_name']}<br>" +
                f"To: {route['end_station_name']}<br>" +
                f"Number of rides: {route['rides']:,}<br>" +
                "<extra></extra>"
            ),
            showlegend=False
        ))
    
    # Add a small delay to avoid hitting API rate limits
    time.sleep(1)

# Add station points
stations = pd.concat([
    df_2024[['start_station_name', 'start_lat', 'start_lng']].rename(
        columns={'start_station_name': 'station_name', 'start_lat': 'lat', 'start_lng': 'lng'}
    ),
    df_2024[['end_station_name', 'end_lat', 'end_lng']].rename(
        columns={'end_station_name': 'station_name', 'end_lat': 'lat', 'end_lng': 'lng'}
    )
]).drop_duplicates()

fig.add_trace(go.Scattermapbox(
    lat=stations['lat'],
    lon=stations['lng'],
    mode='markers',
    marker=dict(
        size=4,
        color='white',
        opacity=0.8
    ),
    showlegend=False,
    hoverinfo='none'
))

# Previous code remains the same until the layout section

# Previous code remains the same until the layout section

# Create a better color gradient for the routes
def get_route_color(normalized_value):
    if normalized_value < 0.25:
        return f'rgba(65, 105, 225, 0.7)'  # Blue for low usage
    elif normalized_value < 0.5:
        return f'rgba(34, 139, 34, 0.7)'   # Green for medium usage
    elif normalized_value < 0.75:
        return f'rgba(255, 140, 0, 0.7)'   # Orange for high usage
    else:
        return f'rgba(255, 0, 0, 0.7)'     # Red for maximum usage

# Modify the route traces
for idx, route in top_routes.iterrows():
    coords = get_route_coordinates(
        route['start_lng'], 
        route['start_lat'], 
        route['end_lng'], 
        route['end_lat']
    )
    
    if coords:
        lngs, lats = zip(*coords)
        
        # Get color based on number of rides
        route_color = get_route_color(route['normalized_rides'])
        
        fig.add_trace(go.Scattermapbox(
            mode='lines',
            lon=list(lngs),
            lat=list(lats),
            line=dict(
                width=3,
                color=route_color
            ),
            hovertemplate=(
                f"<b>Route {idx+1} of 300</b><br>" +
                f"From: {route['start_station_name']}<br>" +
                f"To: {route['end_station_name']}<br>" +
                f"Number of rides: {route['rides']:,}<br>" +
                "<extra></extra>"
            ),
            showlegend=False
        ))
    
    time.sleep(1)

# Update layout with modified color scale
fig.update_layout(
    mapbox_style="carto-darkmatter",
    mapbox=dict(
        center=dict(lat=40.7505, lon=-73.9934),
        zoom=11.5
    ),
    margin={"r":0,"t":50,"l":0,"b":0},
    title={
        'text': "Most Popular Citibike Routes (2024)",
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(color='#333333', size=16)
    },
    paper_bgcolor='#1a1f35',
    plot_bgcolor='#1a1f35'
)

# Add a separate trace for the colorbar with new color scale
fig.add_trace(go.Scattermapbox(
    lat=[None],
    lon=[None],
    mode='markers',
    marker=dict(
        size=0,
        colorscale=[
            [0, 'rgb(65, 105, 225)'],    # Blue
            [0.33, 'rgb(34, 139, 34)'],  # Green
            [0.66, 'rgb(255, 140, 0)'],  # Orange
            [1, 'rgb(255, 0, 0)']        # Red
        ],
        colorbar=dict(
            title='Number of Rides',
            titleside='right',
            x=1.02,
            xanchor='left',
            titlefont=dict(color='white'),
            tickfont=dict(color='white'),
            thickness=15,
            ticktext=[
                f'{int(top_routes["rides"].min()):,}',
                f'{int(top_routes["rides"].mean()):,}',
                f'{int(top_routes["rides"].max()):,}'
            ],
            tickvals=[
                top_routes['rides'].min(),
                top_routes['rides'].mean(),
                top_routes['rides'].max()
            ]
        ),
        cmin=top_routes['rides'].min(),
        cmax=top_routes['rides'].max(),
        showscale=True
    ),
    showlegend=False,
    hoverinfo='none'
))

# Rest of the code remains the same


# Save the map
fig.write_html("site/charts/top_routes_map2.html")

# Print some statistics
print(f"Most popular route: {top_routes.iloc[0]['start_station_name']} → {top_routes.iloc[0]['end_station_name']}")
print(f"Number of rides on most popular route: {top_routes.iloc[0]['rides']:,}")
print(f"Minimum rides shown: {top_routes.iloc[-1]['rides']:,}")


Error: HTTPSConnectionPool(host='api.openrouteservice.org', port=443): Max retries exceeded with url: /v2/directions/cycling-regular/geojson (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000023544F13F80>, 'Connection to api.openrouteservice.org timed out. (connect timeout=None)'))
Error: HTTPSConnectionPool(host='api.openrouteservice.org', port=443): Max retries exceeded with url: /v2/directions/cycling-regular/geojson (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000023544F94200>: Failed to resolve 'api.openrouteservice.org' ([Errno 11001] getaddrinfo failed)"))
Error: HTTPSConnectionPool(host='api.openrouteservice.org', port=443): Read timed out. (read timeout=None)
Error: HTTPSConnectionPool(host='api.openrouteservice.org', port=443): Max retries exceeded with url: /v2/directions/cycling-regular/geojson (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000023544F965D0>: Failed to reso

#### Main routes - option4 - color gradient

In [17]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

In [18]:
os.chdir("C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data")

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import requests
import time

# Your OpenRouteService API key
ORS_API_KEY = ors_api_key

def get_route_coordinates(start_lng, start_lat, end_lng, end_lat):
    """Get cycling route from OpenRouteService API"""
    headers = {
        'Authorization': ORS_API_KEY,
        'Content-Type': 'application/json; charset=utf-8'
    }
    
    body = {"coordinates": [[start_lng, start_lat], [end_lng, end_lat]],
            "profile": "cycling-regular",
            "format": "geojson"}
    
    try:
        response = requests.post(
            'https://api.openrouteservice.org/v2/directions/cycling-regular/geojson',
            json=body,
            headers=headers
        )
        
        if response.status_code == 200:
            route = response.json()
            return route['features'][0]['geometry']['coordinates']
        else:
            print(f"Error getting route: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

def get_color(normalized_value):
    """
    Returns color based on normalized value:
    1.0 -> Red (most popular)
    0.5 -> Purple (medium popular)
    0.0 -> Blue (least popular)
    """
    if normalized_value > 0.7:
        # Red range
        intensity = min(max((normalized_value - 0.7) / 0.3, 0), 1)
        return f'rgba(255, 50, 50, 0.8)'
    elif normalized_value > 0.3:
        # Purple range
        intensity = min(max((normalized_value - 0.3) / 0.4, 0), 1)
        return f'rgba(180, 50, 180, 0.7)'
    else:
        # Blue range
        intensity = min(max(normalized_value / 0.3, 0), 1)
        return f'rgba(50, 50, 255, 0.6)'


def get_top_routes(df_2024, top_n=200):
    routes = df_2024.groupby(
        ['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
    ).size().reset_index(name='rides')
    
    routes = routes[routes['start_station_name'] != routes['end_station_name']]
    return routes.nlargest(top_n, 'rides')

# Get top 1000 routes
top_routes = get_top_routes(df_2024, top_n=1000)

# Normalize rides for color scaling
top_routes['normalized_rides'] = (top_routes['rides'] - top_routes['rides'].min()) / \
                                (top_routes['rides'].max() - top_routes['rides'].min())

# Create figure
fig = go.Figure()

# Add routes with actual paths
for idx, route in top_routes.iterrows():
    coords = get_route_coordinates(
        route['start_lng'], 
        route['start_lat'], 
        route['end_lng'], 
        route['end_lat']
    )
    
    if coords:
        lngs, lats = zip(*coords)
        
        # Get color based on normalized rides
        route_color = get_color(route['normalized_rides'])
        
        fig.add_trace(go.Scattermapbox(
            mode='lines',
            lon=list(lngs),
            lat=list(lats),
            line=dict(
                width=2,  # Reduced width for better visibility with more routes
                color=route_color
            ),
            hovertemplate=(
                f"<b>Route {idx+1} of 1000</b><br>" +
                f"From: {route['start_station_name']}<br>" +
                f"To: {route['end_station_name']}<br>" +
                f"Number of rides: {route['rides']:,}<br>" +
                f"Popularity Rank: {idx+1}<br>" +
                "<extra></extra>"
            ),
            showlegend=False
        ))
    
    # Add a small delay to avoid hitting API rate limits
    time.sleep(1)

# Add station points
stations = pd.concat([
    df_2024[['start_station_name', 'start_lat', 'start_lng']].rename(
        columns={'start_station_name': 'station_name', 'start_lat': 'lat', 'start_lng': 'lng'}
    ),
    df_2024[['end_station_name', 'end_lat', 'end_lng']].rename(
        columns={'end_station_name': 'station_name', 'end_lat': 'lat', 'end_lng': 'lng'}
    )
]).drop_duplicates()

# Add a color scale reference
fig.add_trace(go.Scattermapbox(
    lat=[None],
    lon=[None],
    mode='markers',
    marker=dict(
        size=0,
        colorscale=[
            [0, 'rgba(50, 50, 255, 0.6)'],    # Blue
            [0.3, 'rgba(155, 50, 255, 0.7)'],  # Purple
            [0.7, 'rgba(255, 50, 50, 0.8)']    # Red
        ],
        colorbar=dict(
            title='Number of Rides',
            titleside='right',
            x=0.98,
            y=0.5,
            thickness=20,
            len=0.5,
            tickformat=',d',
            tickvals=[0, 0.5, 1],
            ticktext=[f"{int(top_routes['rides'].min()):,}", 
                     f"{int((top_routes['rides'].min() + top_routes['rides'].max())/2):,}", 
                     f"{int(top_routes['rides'].max()):,}"]
        )
    ),
    showlegend=False
))

# Add station markers
fig.add_trace(go.Scattermapbox(
    lat=stations['lat'],
    lon=stations['lng'],
    mode='markers',
    marker=dict(
        size=3,  # Reduced size for better visibility with more routes
        color='white',
        opacity=0.6
    ),
    showlegend=False,
    hoverinfo='none'
))

# Update layout
fig.update_layout(
    mapbox_style="carto-darkmatter",
    mapbox=dict(
        center=dict(lat=40.7505, lon=-73.9934),
        zoom=11.5
    ),
    margin={"r":0,"t":50,"l":0,"b":0},
    title={
        'text': "Top 1000 Most Popular Citibike Routes (2024)<br>" +
               "<sup>Red = Most Popular → Purple → Blue = Less Popular</sup>",
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(color='white', size=16)
    },
    paper_bgcolor='#1a1f35',
    plot_bgcolor='#1a1f35'
)

# Save the map
fig.write_html("site/charts/top_routes_map1.html")

# Print some statistics
print(f"Most popular route: {top_routes.iloc[0]['start_station_name']} → {top_routes.iloc[0]['end_station_name']}")
print(f"Number of rides on most popular route: {top_routes.iloc[0]['rides']:,}")
print(f"Minimum rides shown: {top_routes.iloc[-1]['rides']:,}")

Error: HTTPSConnectionPool(host='api.openrouteservice.org', port=443): Read timed out. (read timeout=None)
Most popular route: River Ter & Warren St → Vesey Pl & River Terrace
Number of rides on most popular route: 7,367
Minimum rides shown: 1,362


### 4. Main routes - color coded

In [19]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import requests
import time
from tqdm import tqdm

# Mapbox access token - free to generate
MAPBOX_TOKEN = mapbox_token_key  # Get from https://account.mapbox.com/

def get_route_coordinates(start_lng, start_lat, end_lng, end_lat):
    """Get cycling route from Mapbox Directions API"""
    url = f"https://api.mapbox.com/directions/v5/mapbox/cycling/{start_lng},{start_lat};{end_lng},{end_lat}"
    params = {
        "access_token": MAPBOX_TOKEN,
        "geometries": "geojson",
        "overview": "full"
    }
    
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            route = response.json()
            if route["routes"]:
                return route["routes"][0]["geometry"]["coordinates"]
        return None
    except Exception as e:
        print(f"Error getting route: {e}")
        return None

def get_top_routes(df_2024, top_n=500):
    """Get top n routes"""
    routes = df_2024.groupby(
        ['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
    ).size().reset_index(name='rides')
    
    routes = routes[routes['start_station_name'] != routes['end_station_name']]
    return routes.nlargest(top_n, 'rides')

def get_route_color(normalized_value):
    """Get color based on normalized value"""
    if normalized_value < 0.25:
        return f'rgba(0,114,206,0.7)'    # Blue
    elif normalized_value < 0.5:
        return f'rgba(255,255,0,0.7)'    # Yellow
    elif normalized_value < 0.75:
        return f'rgba(255,140,0,0.7)'    # Orange
    else:
        return f'rgba(255,0,0,0.7)'      # Red

# Get top routes
print("Getting top routes...")
top_routes = get_top_routes(df_2024, top_n=500)

# Normalize rides for color scaling
top_routes['normalized_rides'] = (top_routes['rides'] - top_routes['rides'].min()) / \
                                (top_routes['rides'].max() - top_routes['rides'].min())

# Create figure
fig = go.Figure()

print("Getting and plotting routes...")
# Add routes with actual paths
for idx, route in tqdm(top_routes.iterrows(), total=len(top_routes)):
    coords = get_route_coordinates(
        route['start_lng'],
        route['start_lat'],
        route['end_lng'],
        route['end_lat']
    )
    
    if coords:
        # Extract lons and lats from coords
        lons, lats = zip(*coords)
        
        fig.add_trace(go.Scattermapbox(
            mode='lines',
            lon=list(lons),
            lat=list(lats),
            line=dict(
                width=3 + (route['normalized_rides'] * 4),
                color=get_route_color(route['normalized_rides'])
            ),
            hovertemplate=(
                f"<b>Route {idx+1} of 100</b><br>" +
                f"From: {route['start_station_name']}<br>" +
                f"To: {route['end_station_name']}<br>" +
                f"Number of rides: {route['rides']:,}<br>" +
                "<extra></extra>"
            ),
            hoverlabel=dict(
                bgcolor='rgba(26, 31, 53, 0.9)',
                bordercolor='rgba(255, 255, 255, 0.3)',
                font=dict(size=14, color='white')
            ),
            showlegend=False
        ))
    
    # Add small delay to respect API rate limits
    time.sleep(0.1)

# Add station points
stations = pd.concat([
    df_2024[['start_station_name', 'start_lat', 'start_lng']].rename(
        columns={'start_station_name': 'station_name', 'start_lat': 'lat', 'start_lng': 'lng'}
    ),
    df_2024[['end_station_name', 'end_lat', 'end_lng']].rename(
        columns={'end_station_name': 'station_name', 'end_lat': 'lat', 'end_lng': 'lng'}
    )
]).drop_duplicates()

# Add stations to map
fig.add_trace(go.Scattermapbox(
    lat=stations['lat'],
    lon=stations['lng'],
    mode='markers',
    marker=dict(size=4, color='white', opacity=0.8),
    showlegend=False,
    hoverinfo='none'
))

# Update layout
fig.update_layout(
    mapbox=dict(
        accesstoken=MAPBOX_TOKEN,
        style="dark",
        center=dict(lat=40.7505, lon=-73.9934),
        zoom=11.5
    ),
    margin={"r":0,"t":50,"l":0,"b":0},
    title={
        'text': "Most Popular Citibike Routes (2024)",
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(color='#00fff2', size=24)
    },
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

# Add colorbar
fig.add_trace(go.Scattermapbox(
    lat=[None],
    lon=[None],
    mode='markers',
    marker=dict(
        size=0,
        colorscale=[
            [0, 'rgb(0,114,206)'],     # Blue
            [0.33, 'rgb(255,255,0)'],  # Yellow
            [0.66, 'rgb(255,140,0)'],  # Orange
            [1, 'rgb(255,0,0)']        # Red
        ],
        colorbar=dict(
            title='Number of Rides',
            titleside='right',
            x=1.02,
            xanchor='left',
            titlefont=dict(color='white'),
            tickfont=dict(color='white'),
            thickness=15,
            ticktext=[
                f'{int(top_routes["rides"].min()):,}',
                f'{int(top_routes["rides"].mean()):,}',
                f'{int(top_routes["rides"].max()):,}'
            ],
            tickvals=[
                top_routes['rides'].min(),
                top_routes['rides'].mean(),
                top_routes['rides'].max()
            ],
            bgcolor='rgba(0,0,0,0)',
            outlinewidth=0
        ),
        cmin=top_routes['rides'].min(),
        cmax=top_routes['rides'].max(),
        showscale=True
    ),
    showlegend=False,
    hoverinfo='none'
))

print("Saving map...")
# Save the map
fig.write_html("site/charts/top_routes_map6.html",
               full_html=False,
               include_plotlyjs='cdn')

Getting top routes...
Getting and plotting routes...


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [10:34<00:00,  1.27s/it]


Saving map...


#### 5. Color coded map (Map box) correct theme

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import requests
import time
from tqdm import tqdm

# Mapbox access token - free to generate
MAPBOX_TOKEN = mapbox_token_key  # Get from https://account.mapbox.com/

def get_route_coordinates(start_lng, start_lat, end_lng, end_lat):
    """Get cycling route from Mapbox Directions API"""
    url = f"https://api.mapbox.com/directions/v5/mapbox/cycling/{start_lng},{start_lat};{end_lng},{end_lat}"
    params = {
        "access_token": MAPBOX_TOKEN,
        "geometries": "geojson",
        "overview": "full"
    }
    
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            route = response.json()
            if route["routes"]:
                return route["routes"][0]["geometry"]["coordinates"]
        return None
    except Exception as e:
        print(f"Error getting route: {e}")
        return None

def get_top_routes(df_2024, top_n=500):
    """Get top n routes"""
    routes = df_2024.groupby(
        ['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
    ).size().reset_index(name='rides')
    
    routes = routes[routes['start_station_name'] != routes['end_station_name']]
    return routes.nlargest(top_n, 'rides')

def get_route_color(normalized_value):
    """Get color based on normalized value"""
    if normalized_value < 0.25:
        return f'rgba(0,114,206,0.7)'    # Blue
    elif normalized_value < 0.5:
        return f'rgba(255,255,0,0.7)'    # Yellow
    elif normalized_value < 0.75:
        return f'rgba(255,140,0,0.7)'    # Orange
    else:
        return f'rgba(255,0,0,0.7)'      # Red

# Get top routes
print("Getting top routes...")
top_routes = get_top_routes(df_2024, top_n=500)

# Normalize rides for color scaling
top_routes['normalized_rides'] = (top_routes['rides'] - top_routes['rides'].min()) / \
                                (top_routes['rides'].max() - top_routes['rides'].min())



In [30]:
# Create figure
fig = go.Figure()

print("Getting and plotting routes...")
# Add routes with actual paths
for idx, route in tqdm(top_routes.iterrows(), total=len(top_routes)):
    coords = get_route_coordinates(
        route['start_lng'],
        route['start_lat'],
        route['end_lng'],
        route['end_lat']
    )
    
    if coords:
        # Extract lons and lats from coords
        lons, lats = zip(*coords)
        
        fig.add_trace(go.Scattermapbox(
            mode='lines',
            lon=list(lons),
            lat=list(lats),
            line=dict(
                width=3 + (route['normalized_rides'] * 4),
                color=get_route_color(route['normalized_rides'])
            ),
            hovertemplate=(
                f"<b>Route {idx+1} of 100</b><br>" +
                f"From: {route['start_station_name']}<br>" +
                f"To: {route['end_station_name']}<br>" +
                f"Number of rides: {route['rides']:,}<br>" +
                "<extra></extra>"
            ),
            hoverlabel=dict(
                bgcolor='rgba(26, 31, 53, 0.9)',
                bordercolor='rgba(255, 255, 255, 0.3)',
                font=dict(size=14, color='white')
            ),
            showlegend=False
        ))
    
    # Add small delay to respect API rate limits
    time.sleep(0.1)

# Add station points
stations = pd.concat([
    df_2024[['start_station_name', 'start_lat', 'start_lng']].rename(
        columns={'start_station_name': 'station_name', 'start_lat': 'lat', 'start_lng': 'lng'}
    ),
    df_2024[['end_station_name', 'end_lat', 'end_lng']].rename(
        columns={'end_station_name': 'station_name', 'end_lat': 'lat', 'end_lng': 'lng'}
    )
]).drop_duplicates()

# Add stations to map
fig.add_trace(go.Scattermapbox(
    lat=stations['lat'],
    lon=stations['lng'],
    mode='markers',
    marker=dict(size=4, color='white', opacity=0.8),
    showlegend=False,
    hoverinfo='none'
))

# ... (previous code remains same until layout section)

# Update layout
fig.update_layout(
    mapbox=dict(
        style="carto-darkmatter",  # Changed to carto-darkmatter
        center=dict(lat=40.7505, lon=-73.9934),
        zoom=11.5
    ),
    margin={"r":0,"t":50,"l":0,"b":0},
    title={
        'text': "Most Popular Citibike Routes (2024)",
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(color='#00fff2', size=24)
    },
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

# Add colorbar with updated styling
fig.add_trace(go.Scattermapbox(
    lat=[None],
    lon=[None],
    mode='markers',
    marker=dict(
        size=0,
        colorscale=[
            [0, 'rgb(0,114,206)'],     # Blue
            [0.33, 'rgb(255,255,0)'],  # Yellow
            [0.66, 'rgb(255,140,0)'],  # Orange
            [1, 'rgb(255,0,0)']        # Red
        ],
        colorbar=dict(
            title=dict(
                text='Number of Rides',
                font=dict(color='#00fff2')  # Changed to cyan
            ),
            titleside='right',
            x=1.02,
            xanchor='left',
            titlefont=dict(color='#00fff2'),  # Changed to cyan
            tickfont=dict(color='white'),
            thickness=15,
            ticktext=[
                f'{int(top_routes["rides"].min()):,}',
                f'{int(top_routes["rides"].mean()):,}',
                f'{int(top_routes["rides"].max()):,}'
            ],
            tickvals=[
                top_routes['rides'].min(),
                top_routes['rides'].mean(),
                top_routes['rides'].max()
            ],
            bgcolor='rgba(0,0,0,0)',
            outlinewidth=0
        ),
        cmin=top_routes['rides'].min(),
        cmax=top_routes['rides'].max(),
        showscale=True
    ),
    showlegend=False,
    hoverinfo='none'
))

# ... (rest of the code remains same)


print("Saving map...")
# Save the map
fig.write_html("site/charts/top_routes_map7.html",
               full_html=False,
               include_plotlyjs='cdn')

Getting and plotting routes...


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [11:09<00:00,  1.34s/it]


Saving map...
